In [2]:
import os
import numpy as np
import pandas as pd
from scipy.stats import rankdata
import pprint
from collections import defaultdict

In [9]:
corr_matrix_dir = '/mnt/c/Users/donna/Downloads/Thesis/correlation_results/scrna/control/abs_cors/cov_spread/atac_match'

In [11]:
##Niet lopen over fragment features
# Load all correlation matrices
correlation_matrices = {}
for file in os.listdir(corr_matrix_dir):
    if file.endswith('.csv'):
        tissue_name = os.path.splitext(file)[0].lower()
        matrix = pd.read_csv(os.path.join(corr_matrix_dir, file), index_col=0)
        correlation_matrices[tissue_name] = matrix

# Rank all cell types across tissues within each sample
sample_wise_corrs = {}
for tissue_name, matrix in correlation_matrices.items():
    for sample_name in matrix.columns:  # ← samples are columns
        for cell_type in matrix.index:  # ← cell types are rows
            value = abs(matrix.loc[cell_type, sample_name])
            if not np.isnan(value):
                sample_wise_corrs.setdefault(sample_name, []).append(
                    (cell_type, tissue_name, value)
                )

for sample_name, items in sample_wise_corrs.items():
    print(f"Sample: {sample_name}")
    print(f"  Number of cell types considered (including duplicates): {len(items)}")

    unique_celltypes = set((ct, t) for ct, t, _ in items)
    print(f"  Unique (cell_type, tissue) combinations: {len(unique_celltypes)}")

    print("---")

# Compute and collect rank data
rank_data = []
for sample_name, items in sample_wise_corrs.items():
    cell_types, tissues, values = zip(*items)  # unpack all three fields
    values = np.array(values)
    ranks = rankdata(values, method='ordinal')
    ranks = len(ranks) + 1 - ranks  # flip: highest correlation = rank 1


    for cell_type, tissue, rank in zip(cell_types, tissues, ranks):
        rank_data.append({
            'sample': sample_name,
            'cell_type': f"{cell_type}_{tissue}",
            'tissue': tissue,
            'rank': rank,
        })


df_ranks = pd.DataFrame(rank_data)
output_path = "/mnt/c/Users/donna/Downloads/Thesis/rankjes/cov_spread_ranks_atacmatch.csv"
df_ranks.to_csv(output_path, index=False)


Sample: control--EE88162
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88110
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88136
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88121
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88133
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88107
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88103
  Number of cell types considered (including duplicates): 112
  Unique (cell_type, tissue) combinations: 112
---
Sample: control--EE88104
  Number 

In [145]:
tcga = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/TSSClassification/beds/dts_beds/all_peaks.bed")

In [147]:
print(len(tcga))

550202
